*** @Author: Amar Pawar ***

*** @Date: 2021-09-03 ***

*** @Last Modified by: Amar Pawar ***

*** @Last Modified time: 2021-09-03 ***

*** @Title : AWS EMR+PySpark ***

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('operations').getOrCreate()
sc=spark.sparkContext
from pyspark.sql import *
from pyspark.sql.functions import *
import datetime
sc.setLogLevel('ERROR')

In [ ]:
file_data_df = spark.read.format("csv").option("header", "true").load("s3://amartest1/*.csv")

In [3]:
user_data = file_data_df.groupBy("user_name").count()
user_data.show()

+--------------------+-----+
|           user_name|count|
+--------------------+-----+
|salinabodale73@gm...|  569|
|sharlawar77@gmail...|  580|
|rahilstar11@gmail...|  551|
|deepshukla292@gma...|  565|
|  iamnzm@outlook.com|  614|
|markfernandes66@g...|  508|
|damodharn21@gmail...|  253|
|bhagyashrichalke2...|  482|
+--------------------+-----+



In [4]:
user_data7 = file_data_df.select("user_name").where((file_data_df.keyboard != 0.0) | (file_data_df.mouse != 0.0))
user_data8 = user_data7.groupBy("user_name").count()
work_seconds = user_data8.withColumn('work_seconds',  ( ( (user_data8['count'] - 1) * 5) * 60)/ 6)
highest_avg_hour = work_seconds.withColumn("average_hours", concat(
            floor(col("work_seconds") % 86400 / 3600), lit(":"),
            floor((col("work_seconds") % 86400) % 3600 / 60), lit(""),
           
        ))\
    .drop("work_seconds")
highest_avg_hour.createOrReplaceTempView("sql_view")
highest_avg_hour1 = spark.sql("SELECT user_name, average_hours FROM sql_view ORDER BY average_hours desc")
highest_avg_hour.show()

2021-09-02 16:03:42,369 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----+-------------+
|           user_name|count|average_hours|
+--------------------+-----+-------------+
|salinabodale73@gm...|  440|          6:5|
|sharlawar77@gmail...|  457|         6:20|
|rahilstar11@gmail...|  399|         5:31|
|deepshukla292@gma...|  475|         6:35|
|  iamnzm@outlook.com|  459|         6:21|
|markfernandes66@g...|  389|         5:23|
|damodharn21@gmail...|  191|         2:38|
|bhagyashrichalke2...|  361|          5:0|
+--------------------+-----+-------------+



In [5]:
highest_avg_hour.createOrReplaceTempView("sql_view1")
lowest_avg_hour = spark.sql("SELECT user_name, count, average_hours FROM sql_view1 ORDER BY average_hours asc")
lowest_avg_hour.show()

+--------------------+-----+-------------+
|           user_name|count|average_hours|
+--------------------+-----+-------------+
|damodharn21@gmail...|  191|         2:38|
|bhagyashrichalke2...|  361|          5:0|
|markfernandes66@g...|  389|         5:23|
|rahilstar11@gmail...|  399|         5:31|
|sharlawar77@gmail...|  457|         6:20|
|  iamnzm@outlook.com|  459|         6:21|
|deepshukla292@gma...|  475|         6:35|
|salinabodale73@gm...|  440|          6:5|
+--------------------+-----+-------------+



In [6]:
columns1 = ['user_name', 'keyboard', 'mouse']
user_data5 = file_data_df.select("user_name","keyboard","mouse").where((file_data_df.keyboard==0)&(file_data_df.mouse==0)).toDF(*columns1)
user_data6 = user_data5.groupBy("user_name").count()
idle_minit = user_data6.withColumn('idle_minits',  ( ((user_data6['count'] - 1) * 5) * 60 / 6) )  
idle_hour = idle_minit.withColumn("idle_hours", concat(
            floor(col("idle_minits") % 86400 / 3600), lit(":"),
            floor((col("idle_minits") % 86400) % 3600 / 60), lit(""),
           
        ))\
    .drop("idle_minits")\
    .sort(desc("idle_hours"))
idle_hour.show(truncate=False)

+----------------------------+-----+----------+
|user_name                   |count|idle_hours|
+----------------------------+-----+----------+
|iamnzm@outlook.com          |155  |2:8       |
|rahilstar11@gmail.com       |152  |2:5       |
|salinabodale73@gmail.com    |133  |1:50      |
|sharlawar77@gmail.com       |123  |1:41      |
|bhagyashrichalke21@gmail.com|121  |1:40      |
|markfernandes66@gmail.com   |119  |1:38      |
|deepshukla292@gmail.com     |90   |1:14      |
|damodharn21@gmail.com       |62   |0:50      |
+----------------------------+-----+----------+

